Setup

In [32]:
#Import libraries needed 
import pandas as pd
from sqlalchemy import create_engine
pd.set_option('display.max_columns', None)  #no column limit




In [33]:
#Load data from the CSV file
df = pd.read_csv("/Users/martinsvitek/Desktop/onboarding webeet/_onboarding_data/daily_tasks/day_4/day_4_datasets/sat-results.csv")
df.head() 

,DBN,SCHOOL NAME,Num of SAT Test Takers,SAT Critical Reading Avg. Score,SAT Math Avg. Score,SAT Writing Avg. Score,SAT Critical Readng Avg. Score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0


In [34]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DBN                              493 non-null    object 
 1   SCHOOL NAME                      493 non-null    object 
 2   Num of SAT Test Takers           493 non-null    object 
 3   SAT Critical Reading Avg. Score  493 non-null    object 
 4   SAT Math Avg. Score              493 non-null    object 
 5   SAT Writing Avg. Score           493 non-null    object 
 6   SAT Critical Readng Avg. Score   493 non-null    object 
 7   internal_school_id               493 non-null    int64  
 8   contact_extension                388 non-null    object 
 9   pct_students_tested              376 non-null    object 
 10  academic_tier_rating             402 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 42.5+ KB


,internal_school_id,academic_tier_rating
count,493.000000,402.000000
mean,562172.943205,2.564677
std,262138.627055,1.126443
min,101855.000000,1.000000
25%,332013.000000,2.000000
50%,587220.000000,3.000000
75%,782993.000000,4.000000
max,999398.000000,4.000000


 <h2>Clean the data</h2>	


In [35]:
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(r"\s+", "_", regex=True)       #any whitespace to underscore
      .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)  #strip special chars
)
df

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,sat_critical_readng_avg_score,internal_school_id,contact_extension,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355,404,363,355,218160,x345,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383,423,366,383,268547,x234,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377,402,370,377,236446,x123,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414,401,359,414,427826,x123,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390,433,384,390,672714,x123,92%,2.0
...,...,...,...,...,...,...,...,...,...,...,...
488,27Q480,JOHN ADAMS HIGH SCHOOL,403,391,409,392,391,863765,NaN,92%,1.0
489,13K605,GEORGE WESTINGHOUSE CAREER AND TECHNICAL EDUCA...,85,406,391,392,406,937579,x234,NaN,NaN
490,05M304,MOTT HALL HIGH SCHOOL,54,413,399,398,413,296405,x123,78%,2.0
491,02M520,MURRY BERGTRAUM HIGH SCHOOL FOR BUSINESS CAREERS,264,407,440,393,407,892839,NaN,92%,2.0


 <h2>Drop columsn not needed</h2>	


In [36]:
df = df.drop(columns=[
    "sat_critical_readng_avg_score",  #just a typo duplicate 
    "internal_school_id",             #we have dbn as the unique key
    "contact_extension"               #many nulls, not really useful
])

convert dtype

In [37]:
#Convert number of test takers to integer and replace non numeric values with NaN
df["num_of_sat_test_takers"] = pd.to_numeric(df["num_of_sat_test_takers"], errors="coerce").astype('Int64')

In [38]:
#Convert numeric scores columns to float type and replace non numeric values with NaN
score_columns = [
    "sat_critical_reading_avg_score",
    "sat_math_avg_score",
    "sat_writing_avg_score"
]
df[score_columns] = df[score_columns].apply(pd.to_numeric, errors="coerce")

df.head() 

,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,pct_students_tested,academic_tier_rating
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,78%,2.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,NaN,3.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,NaN,3.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,92%,4.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,92%,2.0


In [39]:
score_cols = [
    "sat_critical_reading_avg_score",
    "sat_math_avg_score",
    "sat_writing_avg_score"
]

df[score_cols].agg(['min', 'max'])


,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score
min,279.0,-10.0,286.0
max,679.0,1100.0,682.0


In [40]:
# delete %, convert to float, set "N/A" as NaN
df["pct_students_tested"] = (
    df["pct_students_tested"]
    .replace("N/A", None)
    .str.replace("%", "", regex=False)
    .astype(float)
)

In [41]:
# Convert data type to integer for tier rating 
df["academic_tier_rating"] = pd.to_numeric(df["academic_tier_rating"], errors="coerce").astype('Int64')

 <h2>add usefull columns</h2>	


In [42]:
#Add average total score (mean of 3 sections)
df['sat_avg_total_score'] = (
    df[score_cols].mean(axis=1)
    .round()
    .astype('Int64')  # nullable integer dtype keeps <NA>
)
df.head()


,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,pct_students_tested,academic_tier_rating,sat_avg_total_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,78.0,2,374
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,NaN,3,391
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,NaN,3,383
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,92.0,4,391
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,92.0,2,402


In [43]:
# Add sum total score (sum of 3 sections)
df['sat_sum_total_score'] = df[score_cols].sum(axis=1)


In [45]:
#checking

print(df.dtypes)
print(df.head())

dbn                                object
school_name                        object
num_of_sat_test_takers              Int64
sat_critical_reading_avg_score    float64
sat_math_avg_score                float64
sat_writing_avg_score             float64
pct_students_tested               float64
academic_tier_rating                Int64
sat_avg_total_score                 Int64
sat_sum_total_score               float64
dtype: object
      dbn                                    school_name  \
0  01M292  HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES   
1  01M448            UNIVERSITY NEIGHBORHOOD HIGH SCHOOL   
2  01M450                     EAST SIDE COMMUNITY SCHOOL   
3  01M458                      FORSYTH SATELLITE ACADEMY   
4  01M509                        MARTA VALLE HIGH SCHOOL   

   num_of_sat_test_takers  sat_critical_reading_avg_score  sat_math_avg_score  \
0                      29                           355.0               404.0   
1                      91                  

<h2>save cleaned file and integration</h2>	 	



In [46]:
# Save cleaned version
df.to_csv("sat_results_cleaned.csv", index=False)

In [47]:
import os
print(os.getcwd())  # current folder


/Users/martinsvitek/Desktop/onboarding webeet/_onboarding_data/daily_tasks/day_4


In [48]:
# SQLAlchemy connection 

DATABASE_URL = (
    "postgresql+psycopg2://neondb_owner:npg_CeS9fJg2azZD"
    "@ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech:5432/neondb"
    "?sslmode=require"
)

# Create engine and establish connection
engine = create_engine(DATABASE_URL)

In [59]:
# Insert into database
df.to_sql("martin_svitek_sat_results", engine, if_exists="replace", index=False)
print("Data uploaded")

#  set dbn as primary key
    
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE martin_svitek_sat_results
        ADD PRIMARY KEY (dbn);
    """))


Data uploaded


IntegrityError: (psycopg2.errors.UniqueViolation) could not create unique index "martin_svitek_sat_results_pkey"
DETAIL:  Key (dbn)=(27Q480) is duplicated.

[SQL: 
        ALTER TABLE martin_svitek_sat_results
        ADD PRIMARY KEY (dbn);
    ]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [60]:
pd.read_sql("""
SELECT dbn, COUNT(*) AS cnt
FROM martin_svitek_sat_results
GROUP BY dbn
HAVING COUNT(*) > 1
ORDER BY cnt DESC;
""", engine)


,dbn,cnt
0,13K605,3
1,27Q480,3
2,07X221,3
3,02M520,3
4,05M304,3
5,14K685,2
6,02M419,2
7,17K543,2
8,09X525,2
9,19K420,2


In [61]:
pd.read_sql("""
SELECT COUNT(*) AS null_dbn
FROM martin_svitek_sat_results
WHERE dbn IS NULL;
""", engine)


,null_dbn
0,0


In [ ]:
# drop duplicates on dbn
df_clean = df.drop_duplicates(subset='dbn')

# write to DB
df_clean.to_sql('martin_svitek_sat_results', engine, if_exists='replace', index=False)

# add PK
from sqlalchemy import text
with engine.begin() as conn:
    conn.execute(text("""
        ALTER TABLE martin_svitek_sat_results
        ADD PRIMARY KEY (dbn);
    """))



<h2>NYC SAT Results Cleaning & Integration
</h2> 

#Cleaning Logic<br>
Standardized column names: lowercase, spaces > underscores, removed special characters.<br>
Dropped unnecessary columns:<br>
sat_critical_readng_avg_score ( typo duplicate)<br>
internal_school_id (DBN already unique id)<br>
contact_extension (mostly null, not relevant)<br>
Converted object to numeric columns in sat, test and tier colums:<br>
Added sat_total_avg_score as the sum.


In [63]:
from sqlalchemy import text

with engine.connect() as con:
    df_preview = pd.read_sql(text("SELECT * FROM martin_svitek_sat_results LIMIT 10;"), con)
df_preview




,dbn,school_name,num_of_sat_test_takers,sat_critical_reading_avg_score,sat_math_avg_score,sat_writing_avg_score,pct_students_tested,academic_tier_rating,sat_avg_total_score,sat_sum_total_score
0,01M292,HENRY STREET SCHOOL FOR INTERNATIONAL STUDIES,29,355.0,404.0,363.0,78.0,2.0,374,1122.0
1,01M448,UNIVERSITY NEIGHBORHOOD HIGH SCHOOL,91,383.0,423.0,366.0,NaN,3.0,391,1172.0
2,01M450,EAST SIDE COMMUNITY SCHOOL,70,377.0,402.0,370.0,NaN,3.0,383,1149.0
3,01M458,FORSYTH SATELLITE ACADEMY,7,414.0,401.0,359.0,92.0,4.0,391,1174.0
4,01M509,MARTA VALLE HIGH SCHOOL,44,390.0,433.0,384.0,92.0,2.0,402,1207.0
5,01M515,LOWER EAST SIDE PREPARATORY HIGH SCHOOL,112,332.0,557.0,316.0,NaN,3.0,402,1205.0
6,01M539,"NEW EXPLORATIONS INTO SCIENCE, TECHNOLOGY AND ...",159,522.0,574.0,525.0,78.0,2.0,540,1621.0
7,01M650,CASCADES HIGH SCHOOL,18,417.0,418.0,411.0,92.0,4.0,415,1246.0
8,01M696,BARD HIGH SCHOOL EARLY COLLEGE,130,624.0,604.0,628.0,NaN,NaN,619,1856.0
9,02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECO...,16,395.0,400.0,387.0,78.0,4.0,394,1182.0


In [64]:
from sqlalchemy import inspect

insp = inspect(engine)
cols = insp.get_columns('martin_svitek_sat_results')
for c in cols:
    print(c['name'], c['type'], "nullable=", c['nullable'])


dbn TEXT nullable= False
school_name TEXT nullable= True
num_of_sat_test_takers BIGINT nullable= True
sat_critical_reading_avg_score DOUBLE PRECISION nullable= True
sat_math_avg_score DOUBLE PRECISION nullable= True
sat_writing_avg_score DOUBLE PRECISION nullable= True
pct_students_tested DOUBLE PRECISION nullable= True
academic_tier_rating BIGINT nullable= True
sat_avg_total_score BIGINT nullable= True
sat_sum_total_score DOUBLE PRECISION nullable= True


- Data inserted using SQLAlchemy  
- DBN used as primary key for relational integrity with other existing tables